<a href="https://colab.research.google.com/github/BVSSandilyan/smart-SDLC-AI/blob/main/Copy_of_smart_SDLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from PIL import Image, ImageDraw  # ✅ Added for image generation

model_name = "ibm-granite/granite-3.3-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

# ✅ Modified to return both AI response and image
def granite_inference_structured(phase, prompt):
    try:
        # Construct a more specific prompt based on the phase and user input
        structured_prompt = f"SDLC Phase: {phase}\nTask/Question: {prompt}\nOutput:"
        # Use the existing tokenizer and model for inference
        if model and tokenizer:
            inputs = tokenizer(structured_prompt, return_tensors="pt").to(model.device)
            outputs = model.generate(
                inputs.input_ids,
                max_new_tokens=1000,
                num_return_sequences=1,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                early_stopping=True
            )
            response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()

            # ✅ Create a simple image based on the input
            img = Image.new("RGB", (600, 100), color=(128, 128, 128))
            draw = ImageDraw.Draw(img)
            display_text = f"{phase}: {prompt[:500]}..."  # Truncated prompt on image
            draw.text((10, 40), display_text, fill="white")

            return response, img
        else:
            return "Model not loaded. Cannot perform inference.", None
    except Exception as e:
        return f"An error occurred during inference: {e}", None

# ✅ Updated to display both AI response and generated image
iface_structured = gr.Interface(
    fn=granite_inference_structured,
    inputs=[
        gr.Dropdown(
            ["Requirements", "Design", "Development", "Testing", "Deployment", "Maintenance", "General/Other"],
            label="Select SDLC Phase"
        ),
        gr.Textbox(lines=5, label="Enter your specific question or task:")
    ],
    outputs=[
        gr.Textbox(label="Generated Output:"),
        gr.Image(label="Generated Prompt-Based Image")
    ],
    title="Smart SDLC - AI Enhanced Software Development Lifecycle (Structured Demo)",
    description="Interact with the ibm-granite/granite-3.3-2b-instruct model for SDLC tasks by selecting a phase.",
css="""
        body {
            background-color: CCFFFF;
            color: #000000;
        }
        .gradio-container {
            background-color: #CCFFCC !important;
            border-radius: 10px;
            padding: 20px;
            box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.05);
        }
        textarea, input, select {
            background-color: #ffffff !important;
            color: #111111 !important;
            font-weight: 500;
            border: 1px solid #cccccc;
            border-radius: 6px;
        }
        button {
            background-color: #1976d2 !important;
            color: white !important;
            font-weight: normal;
            border-radius: 8px;
        }
    """
)

# Launch the structured Gradio interface
print("\nLaunching Structured Gradio Interface...")
iface_structured.launch(share=True)